In [2]:
import torch

n, p = 10, 3
X0 = torch.randn([n, p])

In [4]:
col = 0

In [8]:
B = torch.matmul(X0.T, X0)
B = torch.inv(B)

AttributeError: module 'torch' has no attribute 'inv'

In [3]:
X = X0[:, col]

tensor([[ 0.3721, -0.9717,  0.0511],
        [-0.3526, -0.1424, -1.0795],
        [-0.9186,  0.2163,  0.1396],
        [-2.0612,  3.0459, -1.7796],
        [-2.0841,  1.1208,  0.8691],
        [ 0.9950,  1.5732,  0.3247],
        [-0.9427,  1.4368, -0.0565],
        [-0.4265, -1.2725,  1.3100],
        [-0.6424, -0.4915,  0.5682],
        [-0.3732, -0.0665, -0.8216]])